In [1]:
import pandas as pd
import re
import requests
from sqlalchemy import create_engine, inspect

In [95]:
df_degra = pd.read_csv(r'http://sc-e.fr/docs/degradations_2024-06-12.csv')
df_degra

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,57.508865,2024-06-12,False
1,D009402,E170_6353,REAE170-E170_6353-1,54.499522,2024-06-12,False
2,D000812,E170_6353,SYSE170-E170_6353-2,20.124777,2024-06-12,False
3,D006665,E170_6353,ORDE170-E170_6353-3,10.015077,2024-06-12,False
4,D001163,E170_6353,AUTE170-E170_6353-4,46.814259,2024-06-12,False
...,...,...,...,...,...,...
10525,D006322,CRJ700_2988,SYSCRJ700-CRJ700_2988-40,55.396210,2024-06-08,False
10526,D005497,CRJ700_2988,PORCRJ700-CRJ700_2988-41,18.476565,2024-06-08,False
10527,D005625,CRJ700_2988,HUBCRJ700-CRJ700_2988-42,45.518510,2024-06-08,False
10528,D003657,CRJ700_2988,ECLCRJ700-CRJ700_2988-43,38.398505,2024-06-08,False


In [14]:
import pandas as pd
import re
import requests
from sqlalchemy import create_engine, inspect

# Configurer la connexion à la base de données
db_path = r'C:\Document\Python\Projet\Projet_3\Top_gun\bdd_sql\top_gun_data_1.db'
engine = create_engine(f'sqlite:///{db_path}')
date_defini = '2024-06-16'

# Fonction pour supprimer l'heure, les minutes et les secondes représentant une date et une heure
def supprimer_heure_minutes_secondes(chaine):
    nouvelle_chaine = re.sub(r'\s\d{2}:\d{2}:\d{2}$', '', chaine)
    return nouvelle_chaine

# Fonction pour ne mettre dans la base de données uniquement les measure_day du jour
def measure_today(data, date):
    filtered_data = data[data['measure_day'] == pd.to_datetime(date).date()]
    return filtered_data

# Fonction pour nettoyer et transformer les données de dégradations
def clean_degradations(df):
    # Appliquer la fonction pour supprimer l'heure, les minutes et les secondes
    df['measure_day'] = df['measure_day'].astype(str).apply(supprimer_heure_minutes_secondes)
    
    # Mettre la colonne measure_day au format date sans inclure l'heure
    df['measure_day'] = pd.to_datetime(df['measure_day'], format='%Y-%m-%d', errors='coerce').dt.date

    # Transformer les colonnes spécifiques
    df['need_replacement'] = df['need_replacement'].replace({True: 1, False: 0}).astype(int)
    df['usure_nouvelle'] = df['usure_nouvelle'].round(0)
    
    # Exclure les lignes où 'linked_aero' est 'E170_5551'
    df = df[df['linked_aero'] != 'E170_5551']

    return df

# Vérifier si une table existe déjà dans la base de données
def table_exists(engine, table_name):
    inspector = inspect(engine)
    return inspector.has_table(table_name)

# Fonction pour initialiser la base de données avec le fichier d'origine si la table n'existe pas
def initialize_database_degradations(engine):
    if not table_exists(engine, 'degradations'):
        df_degra = pd.read_csv(r'C:\Document\Python\Projet\Projet_3\Top_gun\Datasets\df_degradations\degradations_2024-06-02.csv')
        
        # Nettoyer les données
        cleaned_data = clean_degradations(df_degra)
        
        # Stocker les données dans la base de données SQL
        cleaned_data.to_sql(name='degradations', con=engine, if_exists='replace', index=False)
        print("Base de données initialisée avec le fichier d'origine pour les dégradations.")
    else:
        print("La table 'degradations' existe déjà, aucune initialisation nécessaire.")

# Fonction pour récupérer les données de dégradations et les stocker dans la base de données SQL
def recup_degradations(date, engine):
    url = f"http://sc-e.fr/docs/degradations_{date}.csv"

    response = requests.get(url)
    data = pd.read_csv(url)

    # Nettoyer les données
    data = clean_degradations(data)

    # Filtrer les données pour ne conserver que celles de la date actuelle
    filtered_data = measure_today(data, date)

    if filtered_data.empty:
        print(f"Aucune donnée à ajouter pour {date}.")
    else:
        # Stocker les données dans la base de données SQL
        filtered_data.to_sql(name='degradations', con=engine, if_exists='append', index=False)
        print(f"Données de dégradations pour {date} ajoutées à la base de données.")

# Appeler la fonction pour initialiser la base de données avec le fichier d'origine si nécessaire
initialize_database_degradations(engine)

# Appeler la fonction pour récupérer et stocker les dégradations du jour
recup_degradations(date_defini, engine)


La table 'degradations' existe déjà, aucune initialisation nécessaire.
Données de dégradations pour 2024-06-16 ajoutées à la base de données.


C:\Users\uvlsp\AppData\Local\Temp\ipykernel_12868\3675021664.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['need_replacement'] = df['need_replacement'].replace({True: 1, False: 0}).astype(int)
